In [1]:
import os
import psycopg2
from psycopg2 import sql, errors
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

True

In [2]:
# Database configuration
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD'),
    'port': int(os.getenv('DB_PORT', '5432'))
}

# Target database name (dimensional model)
TARGET_DB_NAME = os.getenv('TARGET_DB_NAME', 'airbnb_dimensional')

# Validate configuration
if not DB_CONFIG['password']:
    raise ValueError("DB_PASSWORD not set in environment variables")

print("✓ Database configuration loaded:")
print(f"  Host: {DB_CONFIG['host']}")
print(f"  Port: {DB_CONFIG['port']}")
print(f"  User: {DB_CONFIG['user']}")
print(f"  Target Database: {TARGET_DB_NAME}")

✓ Database configuration loaded:
  Host: localhost
  Port: 5432
  User: postgres
  Target Database: airbnb_dimensional


# Create Dimensional Database

**Note**: We connect to the default 'postgres' database first to create our target database.

In [3]:
# Connect to PostgreSQL server (default postgres database)
try:
    # Connect to default database
    conn = psycopg2.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        port=DB_CONFIG['port'],
        database='postgres'
    )
    
    # Set autocommit mode to create database
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = conn.cursor()
    
    # Check if database already exists
    cursor.execute(
        "SELECT 1 FROM pg_database WHERE datname = %s",
        (TARGET_DB_NAME,)
    )
    exists = cursor.fetchone()
    
    if exists:
        print(f"⚠ Database '{TARGET_DB_NAME}' already exists")
        print("  Dropping and recreating to ensure clean state...")
        
        # Terminate existing connections
        cursor.execute(f"""
            SELECT pg_terminate_backend(pg_stat_activity.pid)
            FROM pg_stat_activity
            WHERE pg_stat_activity.datname = '{TARGET_DB_NAME}'
              AND pid <> pg_backend_pid();
        """)
        
        # Drop database
        cursor.execute(sql.SQL("DROP DATABASE {}").format(
            sql.Identifier(TARGET_DB_NAME)
        ))
        print(f"  ✓ Dropped existing database '{TARGET_DB_NAME}'")
    
    # Create database
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(
        sql.Identifier(TARGET_DB_NAME)
    ))
    print(f"✓ Created database '{TARGET_DB_NAME}' successfully")
    
    cursor.close()
    conn.close()
    
except psycopg2.Error as e:
    print(f"✗ Error creating database: {e}")
    raise

⚠ Database 'airbnb_dimensional' already exists
  Dropping and recreating to ensure clean state...
  ✓ Dropped existing database 'airbnb_dimensional'
✓ Created database 'airbnb_dimensional' successfully


# Connect to Dimensional Database


In [4]:
# Connect to the dimensional database
try:
    conn = psycopg2.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        port=DB_CONFIG['port'],
        database=TARGET_DB_NAME
    )
    cursor = conn.cursor()
    
    # Verify connection
    cursor.execute("SELECT version();")
    version = cursor.fetchone()[0]
    print("✓ Connected to dimensional database successfully")
    print(f"  PostgreSQL version: {version.split(',')[0]}")
    
except psycopg2.Error as e:
    print(f"✗ Error connecting to database: {e}")
    raise

✓ Connected to dimensional database successfully
  PostgreSQL version: PostgreSQL 18.0 on x86_64-windows


# Execute Schema SQL

In [5]:
# Read schema SQL file
schema_file = 'database_modelling_schema.sql'

try:
    with open(schema_file, 'r', encoding='utf-8') as f:
        schema_sql = f.read()
    
    print(f"✓ Read schema file: {schema_file}")
    
except FileNotFoundError:
    print(f"✗ Schema file not found: {schema_file}")
    raise

✓ Read schema file: database_modelling_schema.sql


In [6]:
# Execute schema SQL
try:
    print("Executing schema SQL...")
    cursor.execute(schema_sql)
    conn.commit()
    print("✓ Schema created successfully")
    
except psycopg2.Error as e:
    conn.rollback()
    print(f"✗ Error executing schema: {e}")
    raise

Executing schema SQL...
✓ Schema created successfully


# Populate Date Dimension
Includes Calgary-specific seasonality:
- **Stampede** (July): Calgary's famous rodeo and festival
- **Summer Peak** (June, August): High tourism season
- **Winter** (Dec-Feb): Winter season
- **Spring/Fall**: Shoulder seasons

In [7]:
# Call the populate_dim_date function
try:
    cursor.execute("""
        SELECT populate_dim_date('2024-01-01'::DATE, '2026-12-31'::DATE);
    """)
    rows_inserted = cursor.fetchone()[0]
    conn.commit()
    
    print(f"✓ Populated dim_date with {rows_inserted:,} date records")
    print("  Date range: 2024-01-01 to 2026-12-31")
    
except psycopg2.Error as e:
    conn.rollback()
    print(f"✗ Error populating date dimension: {e}")
    raise

✓ Populated dim_date with 1,096 date records
  Date range: 2024-01-01 to 2026-12-31


# Verify Schema Creation

In [8]:
# Get list of all tables
query = """
    SELECT 
        table_name,
        CASE 
            WHEN table_name LIKE 'dim_%' THEN 'Dimension'
            WHEN table_name LIKE 'fact_%' THEN 'Fact'
            WHEN table_name LIKE 'bridge_%' THEN 'Bridge'
            ELSE 'Other'
        END as table_type
    FROM information_schema.tables
    WHERE table_schema = 'public'
        AND table_type = 'BASE TABLE'
    ORDER BY table_type, table_name;
"""

df_tables = pd.read_sql_query(query, conn)
print(f"✓ Total tables created: {len(df_tables)}")
print("\nTables by type:")
print(df_tables.groupby('table_type').size())
print("\nComplete table list:")
df_tables

✓ Total tables created: 9

Tables by type:
table_type
Bridge       1
Dimension    5
Fact         3
dtype: int64

Complete table list:


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\1239405879.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql_query(query, conn)


,table_name,table_type
0,bridge_listing_competitors,Bridge
1,dim_category_ratings,Dimension
2,dim_date,Dimension
3,dim_host,Dimension
4,dim_location,Dimension
5,dim_property,Dimension
6,fact_competitor_pricing_analysis,Fact
7,fact_listing_amenities_summary,Fact
8,fact_listing_metrics,Fact


# Verify Views and Functions


In [9]:
# Get views (including materialized views)
query_views = """
    SELECT 
        table_name as view_name,
        CASE 
            WHEN table_type = 'VIEW' THEN 'View'
            WHEN table_type = 'MATERIALIZED VIEW' THEN 'Materialized View'
        END as view_type
    FROM information_schema.tables
    WHERE table_schema = 'public'
        AND table_type IN ('VIEW', 'MATERIALIZED VIEW')
    ORDER BY view_type, table_name;
"""

df_views = pd.read_sql_query(query_views, conn)
print(f"✓ Views created: {len(df_views)}")
df_views

✓ Views created: 2


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\3504426942.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_views = pd.read_sql_query(query_views, conn)


,view_name,view_type
0,view_listing_summary,View
1,view_price_recommendations,View


In [10]:
# Get functions
query_functions = """
    SELECT 
        routine_name as function_name,
        routine_type as type,
        data_type as return_type
    FROM information_schema.routines
    WHERE routine_schema = 'public'
    ORDER BY routine_name;
"""

df_functions = pd.read_sql_query(query_functions, conn)
print(f"✓ Functions created: {len(df_functions)}")
df_functions

✓ Functions created: 2


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\2058883441.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_functions = pd.read_sql_query(query_functions, conn)


,function_name,type,return_type
0,calculate_distance_km,FUNCTION,numeric
1,populate_dim_date,FUNCTION,integer


# Explore Schema Structure

In [11]:
# Function to display table structure
def show_table_structure(table_name):
    """
    Display the structure of a database table.
    
    Parameters
    ----------
    table_name : str
        Name of the table to describe
    
    Returns
    -------
    pandas.DataFrame
        Table structure with column details
    """
    query = f"""
        SELECT 
            column_name,
            data_type,
            character_maximum_length,
            is_nullable,
            column_default
        FROM information_schema.columns
        WHERE table_name = '{table_name}'
        ORDER BY ordinal_position;
    """
    df = pd.read_sql_query(query, conn)
    print(f"\n{'='*80}")
    print(f"Table: {table_name}")
    print(f"{'='*80}")
    return df

# Show structure of key tables
key_tables = [
    'dim_host',
    'dim_property',
    'dim_location',
    'fact_listing_metrics',
    'bridge_listing_competitors'
]

for table in key_tables:
    display(show_table_structure(table))


Table: dim_host


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\3355479250.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,column_name,data_type,character_maximum_length,is_nullable,column_default
0,host_key,integer,None,NO,nextval('dim_host_host_key_seq'::regclass)
1,host_id,text,None,NO,None
2,host_name,text,None,YES,None
3,host_rating,numeric,None,YES,None
4,host_number_of_reviews,integer,None,YES,0
5,host_response_rate,integer,None,YES,None
6,host_years_hosting,integer,None,YES,None
7,is_superhost,boolean,None,YES,false
8,host_tier,text,None,YES,None
9,experience_level,text,None,YES,None



Table: dim_property


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\3355479250.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


...

C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\3355479250.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,column_name,data_type,character_maximum_length,is_nullable,column_default
0,listing_key,integer,None,NO,nextval('fact_listing_metrics_listing_key_seq'...
1,property_id,text,None,NO,None
2,host_key,integer,None,YES,None
3,property_key,integer,None,YES,None
4,location_key,integer,None,YES,None
5,rating_key,integer,None,YES,None
6,date_key,integer,None,YES,None
7,price_per_night,numeric,None,YES,None
8,listing_rating,numeric,None,YES,None
9,number_of_reviews,integer,None,YES,0



Table: bridge_listing_competitors


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\3355479250.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,column_name,data_type,character_maximum_length,is_nullable,column_default
0,bridge_key,integer,None,NO,nextval('bridge_listing_competitors_bridge_key...
1,listing_key,integer,None,YES,None
2,competitor_listing_key,integer,None,YES,None
3,similarity_rank,integer,None,NO,None
4,overall_similarity_score,numeric,None,NO,None
5,location_similarity,numeric,None,YES,None
6,property_similarity,numeric,None,YES,None
7,quality_similarity,numeric,None,YES,None
8,amenity_similarity,numeric,None,YES,None
9,price_similarity,numeric,None,YES,None


# Verify Date Dimension Data

In [12]:
# Query sample dates showing different seasons
query_dates = """
    SELECT 
        date_key,
        full_date,
        day_name,
        is_weekend,
        season
    FROM dim_date
    WHERE full_date IN (
        '2024-07-15',  -- Stampede
        '2024-08-15',  -- Summer Peak
        '2024-12-25',  -- Winter
        '2024-03-15',  -- Spring/Fall
        '2025-06-01',  -- Summer Peak
        '2025-07-01'   -- Stampede
    )
    ORDER BY full_date;
"""

df_dates = pd.read_sql_query(query_dates, conn)
print("✓ Sample dates from dim_date:")
df_dates

✓ Sample dates from dim_date:


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\1939604008.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dates = pd.read_sql_query(query_dates, conn)


,date_key,full_date,day_name,is_weekend,season
0,20240315,2024-03-15,Friday,False,Spring/Fall
1,20240715,2024-07-15,Monday,False,Stampede
2,20240815,2024-08-15,Thursday,False,Summer Peak
3,20241225,2024-12-25,Wednesday,False,Winter
4,20250601,2025-06-01,Sunday,True,Summer Peak
5,20250701,2025-07-01,Tuesday,False,Stampede


In [13]:
# Show date dimension statistics
query_stats = """
    SELECT 
        MIN(full_date) as start_date,
        MAX(full_date) as end_date,
        COUNT(*) as total_dates,
        COUNT(*) FILTER (WHERE is_weekend = TRUE) as weekend_days,
        COUNT(*) FILTER (WHERE season = 'Stampede') as stampede_days,
        COUNT(*) FILTER (WHERE season = 'Summer Peak') as summer_days,
        COUNT(*) FILTER (WHERE season = 'Winter') as winter_days
    FROM dim_date;
"""

df_stats = pd.read_sql_query(query_stats, conn)
print("✓ Date dimension statistics:")
df_stats

✓ Date dimension statistics:


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\1093877862.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stats = pd.read_sql_query(query_stats, conn)


,start_date,end_date,total_dates,weekend_days,stampede_days,summer_days,winter_days
0,2024-01-01,2026-12-31,1096,312,93,183,271


## 11. Test Utility Functions

Test the Haversine distance calculation function.

**Example**: Calculate distance between Calgary downtown and Calgary Tower.

In [14]:
# Test calculate_distance_km function
# Calgary downtown: 51.0447, -114.0719
# Calgary Tower: 51.0445, -114.0631

query_distance = """
    SELECT 
        calculate_distance_km(51.0447, -114.0719, 51.0445, -114.0631) as distance_km,
        'Calgary Downtown to Calgary Tower' as description;
"""

df_distance = pd.read_sql_query(query_distance, conn)
print("✓ Distance calculation test:")
df_distance

✓ Distance calculation test:


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\1674621960.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_distance = pd.read_sql_query(query_distance, conn)


,distance_km,description
0,0.615609,Calgary Downtown to Calgary Tower


## 12. Schema Creation Summary

Display final summary of the dimensional database.

In [15]:
# Get comprehensive schema summary
print("="*80)
print("DIMENSIONAL DATABASE CREATION COMPLETE")
print("="*80)
print(f"\nDatabase: {TARGET_DB_NAME}")
print(f"Host: {DB_CONFIG['host']}:{DB_CONFIG['port']}")
print(f"\nSchema Components:")
print(f"  • Dimension Tables: {len(df_tables[df_tables['table_type'] == 'Dimension'])}")
print(f"  • Fact Tables: {len(df_tables[df_tables['table_type'] == 'Fact'])}")
print(f"  • Bridge Tables: {len(df_tables[df_tables['table_type'] == 'Bridge'])}")
print(f"  • Views: {len(df_views[df_views['view_type'] == 'View'])}")
print(f"  • Materialized Views: {len(df_views[df_views['view_type'] == 'Materialized View'])}")
print(f"  • Functions: {len(df_functions)}")
print(f"\nDate Dimension:")
print(f"  • Date range: {df_stats['start_date'].iloc[0]} to {df_stats['end_date'].iloc[0]}")
print(f"  • Total dates: {df_stats['total_dates'].iloc[0]:,}")
print("\n" + "="*80)
print("✓ Dimensional database is ready for ETL data loading")
print("="*80)

DIMENSIONAL DATABASE CREATION COMPLETE

Database: airbnb_dimensional
Host: localhost:5432

Schema Components:
  • Dimension Tables: 5
  • Fact Tables: 3
  • Bridge Tables: 1
  • Views: 2
  • Materialized Views: 0
  • Functions: 2

Date Dimension:
  • Date range: 2024-01-01 to 2026-12-31
  • Total dates: 1,096

✓ Dimensional database is ready for ETL data loading


In [16]:
# Import and run the ETL script
import sys
sys.path.append('.')

from etl_normalized_to_dimensional import DimensionalETL

# Configuration (uses same environment variables as above)
source_config = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'database': os.getenv('SOURCE_DB_NAME', 'airbnb_db'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD'),
    'port': int(os.getenv('DB_PORT', '5432'))
}

target_config = {
    'host': DB_CONFIG['host'],
    'database': TARGET_DB_NAME,
    'user': DB_CONFIG['user'],
    'password': DB_CONFIG['password'],
    'port': DB_CONFIG['port']
}

# Run ETL
print("Starting ETL process...")
print("="*80)
etl = DimensionalETL(source_config, target_config)
etl.run_full_etl()
print("\n" + "="*80)
print("ETL completed successfully!")
print("="*80)

2025-11-13 16:01:11,882 - INFO - ======================================================================
2025-11-13 16:01:11,882 - INFO - Starting ETL: Normalized → Dimensional
2025-11-13 16:01:11,883 - INFO - ======================================================================
2025-11-13 16:01:11,933 - INFO - Connected to source database: airbnb_db
2025-11-13 16:01:11,984 - INFO - Connected to target database: airbnb_dimensional
2025-11-13 16:01:11,985 - INFO - 
--- PHASE 1: Loading Dimensions ---
2025-11-13 16:01:11,985 - INFO - Loading dim_host...
2025-11-13 16:01:11,988 - INFO - Extracted 65 hosts from source
2025-11-13 16:01:11,996 - INFO - Loaded 65 hosts into dim_host
2025-11-13 16:01:11,996 - INFO - Loading dim_property...
2025-11-13 16:01:11,999 - INFO - Extracted 100 properties from source
2025-11-13 16:01:12,007 - INFO - Loaded 100 properties into dim_property
2025-11-13 16:01:12,007 - INFO - Loading dim_location...
2025-11-13 16:01:12,008 - INFO - Extracted 86 unique locat

Starting ETL process...


2025-11-13 16:01:13,459 - INFO - Performed K-means clustering with 10 clusters
2025-11-13 16:01:13,472 - INFO - Loaded 86 locations into dim_location
2025-11-13 16:01:13,474 - INFO - Loading dim_category_ratings...
2025-11-13 16:01:13,477 - INFO - Extracted 93 rating sets from source
2025-11-13 16:01:13,486 - INFO - Loaded 93 rating sets into dim_category_ratings
2025-11-13 16:01:13,486 - INFO - 
--- PHASE 2: Loading Central Fact ---
2025-11-13 16:01:13,487 - INFO - Loading fact_listing_metrics...
2025-11-13 16:01:13,488 - INFO - Extracted 100 listings from source
2025-11-13 16:01:13,504 - INFO - Loaded 100 listings into fact_listing_metrics (skipped 0)
2025-11-13 16:01:13,505 - INFO - 
--- PHASE 3: Loading Aggregate Facts ---
2025-11-13 16:01:13,505 - INFO - Loading fact_listing_amenities_summary...
2025-11-13 16:01:13,513 - INFO - Extracted amenities for 100 listings
2025-11-13 16:01:13,530 - INFO - Loaded 100 amenity summaries into fact_listing_amenities_summary
2025-11-13 16:01:13,


ETL completed successfully!


# Verify Data Load

Check that data was successfully loaded into the dimensional database.

In [17]:
# Reconnect to database after ETL
conn = psycopg2.connect(
    host=DB_CONFIG['host'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password'],
    port=DB_CONFIG['port'],
    database=TARGET_DB_NAME
)

# Check row counts for all tables
query_counts = """
    SELECT 
        table_name,
        (xpath('/row/c/text()', query_to_xml(format('select count(*) as c from %I', table_name), false, true, '')))[1]::text::int as row_count
    FROM information_schema.tables
    WHERE table_schema = 'public' 
        AND table_type = 'BASE TABLE'
    ORDER BY table_name;
"""

try:
    df_counts = pd.read_sql_query(query_counts, conn)
    print("✓ Data loaded successfully!\n")
    print("Row counts by table:")
    print("="*50)
    display(df_counts)
    print("="*50)
    print(f"\nTotal rows across all tables: {df_counts['row_count'].sum():,}")
except Exception as e:
    # Fallback if query_to_xml doesn't work
    print("Using alternative counting method...\n")
    
    cursor = conn.cursor()
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
        ORDER BY table_name
    """)
    tables = [row[0] for row in cursor.fetchall()]
    
    results = []
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        results.append({'table_name': table, 'row_count': count})
    
    df_counts = pd.DataFrame(results)
    print("✓ Data loaded successfully!\n")
    print("Row counts by table:")
    print("="*50)
    display(df_counts)
    print("="*50)
    print(f"\nTotal rows across all tables: {df_counts['row_count'].sum():,}")

✓ Data loaded successfully!

Row counts by table:


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\2715850739.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_counts = pd.read_sql_query(query_counts, conn)


,table_name,row_count
0,bridge_listing_competitors,2500
1,dim_category_ratings,93
2,dim_date,1096
3,dim_host,65
4,dim_location,86
5,dim_property,100
6,fact_competitor_pricing_analysis,100
7,fact_listing_amenities_summary,100
8,fact_listing_metrics,100



Total rows across all tables: 4,240


# Sample Queries

Test the dimensional model with some analytical queries.

In [18]:
# Query 1: Top 10 most competitive listings
query_competitive = """
    SELECT 
        f.property_id,
        p.listing_name,
        f.price_per_night,
        f.listing_rating,
        f.number_of_reviews,
        f.competitiveness_score,
        l.location_tier,
        h.host_tier
    FROM fact_listing_metrics f
    JOIN dim_property p ON f.property_key = p.property_key
    JOIN dim_location l ON f.location_key = l.location_key
    JOIN dim_host h ON f.host_key = h.host_key
    ORDER BY f.competitiveness_score DESC
    LIMIT 10;
"""

df_competitive = pd.read_sql_query(query_competitive, conn)
print("Top 10 Most Competitive Listings:")
print("="*80)
display(df_competitive)

Top 10 Most Competitive Listings:


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\3742629631.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_competitive = pd.read_sql_query(query_competitive, conn)


,property_id,listing_name,price_per_night,listing_rating,number_of_reviews,competitiveness_score,location_tier,host_tier
0,48603255,"Entire condo in Calgary, Canada",NaN,4.99,278,64.94,Urban Core,Elite
1,51735826,"Entire condo in Calgary, Canada",NaN,4.98,245,64.88,Downtown Adjacent,Elite
2,551855963871300920,"Entire condo in Calgary, Canada",183.32,4.98,134,64.88,Urban Core,Elite
3,886505572301106555,"Entire condo in Calgary, Canada",192.60,4.98,159,64.88,Urban Core,Elite
4,602404462768217588,"Entire condo in Calgary, Canada",281.76,4.97,153,64.82,Urban Core,Elite
5,847408351544779960,"Entire condo in Calgary, Canada",183.69,4.97,141,64.82,Urban Core,Elite
6,905995983356287446,"Entire condo in Calgary, Canada",167.79,4.96,298,64.76,Urban Core,Elite
7,20457788,"Entire condo in Calgary, Canada",232.00,4.95,121,64.70,Urban Core,Elite
8,648800383850325241,"Entire rental unit in Calgary, Canada",NaN,4.93,166,64.58,Urban Core,Elite
9,949301787438025837,"Entire condo in Calgary, Canada",NaN,4.93,117,64.58,Urban Core,Elite


In [19]:
# Query 2: Sample competitor analysis for a listing
query_competitors = """
    SELECT 
        source_property_id,
        competitor_property_id,
        similarity_rank,
        overall_similarity_score,
        source_price,
        competitor_price,
        ROUND(distance_km, 2) as distance_km
    FROM view_top_competitors
    WHERE listing_key = (SELECT listing_key FROM fact_listing_metrics LIMIT 1)
    ORDER BY similarity_rank
    LIMIT 10;
"""

try:
    df_competitors = pd.read_sql_query(query_competitors, conn)
    print("\nTop 10 Competitors for Sample Listing:")
    print("="*80)
    display(df_competitors)
except Exception as e:
    print(f"Note: Competitor data will be available after ETL completes: {e}")


Top 10 Competitors for Sample Listing:


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\19453031.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_competitors = pd.read_sql_query(query_competitors, conn)


,source_property_id,competitor_property_id,similarity_rank,overall_similarity_score,source_price,competitor_price,distance_km
0,1426378005713860735,917958382556358309,1,97.00,181.5,169.98,0.44
1,1426378005713860735,52441913,2,86.09,181.5,179.00,0.77
2,1426378005713860735,52442502,3,85.75,181.5,179.00,0.80
3,1426378005713860735,1326868547821997402,4,84.85,181.5,140.94,0.48
4,1426378005713860735,551855963871300920,5,83.02,181.5,183.32,1.07
5,1426378005713860735,1364877517827164127,6,82.73,181.5,160.00,1.07
6,1426378005713860735,897769858007491976,7,82.71,181.5,179.50,1.11
7,1426378005713860735,1314598866425258757,8,82.43,181.5,170.11,1.01
8,1426378005713860735,1505895536444280349,9,81.99,181.5,174.67,1.15
9,1426378005713860735,5278986,10,81.74,181.5,123.08,0.68


In [20]:
# Query 3: Price recommendations
query_pricing = """
    SELECT 
        property_id,
        listing_name,
        current_price,
        recommended_optimal_price,
        ROUND(current_price - recommended_optimal_price, 2) as price_difference,
        pricing_status,
        bedrooms,
        location_tier
    FROM view_price_recommendations
    WHERE recommended_optimal_price IS NOT NULL
    ORDER BY ABS(current_price - recommended_optimal_price) DESC
    LIMIT 10;
"""

try:
    df_pricing = pd.read_sql_query(query_pricing, conn)
    print("\nListings with Largest Price Discrepancies:")
    print("="*80)
    display(df_pricing)
except Exception as e:
    print(f"Note: Pricing recommendations will be available after ETL completes: {e}")


Listings with Largest Price Discrepancies:


C:\Users\Ale\AppData\Local\Temp\ipykernel_18456\413664690.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pricing = pd.read_sql_query(query_pricing, conn)


,property_id,listing_name,current_price,recommended_optimal_price,price_difference,pricing_status,bedrooms,location_tier
0,1367726557683973919,"Entire rental unit in Calgary, Canada",None,121.00,None,OPTIMAL,2,Urban Core
1,648800383850325241,"Entire rental unit in Calgary, Canada",None,170.54,None,OPTIMAL,1,Urban Core
2,849869456677351162,"Entire condo in Calgary, Canada",None,133.07,None,OPTIMAL,2,Urban Core
3,1317739365276701089,"Entire rental unit in Calgary, Canada",None,160.88,None,OPTIMAL,1,Urban Core
4,1308401481139155996,"Entire condo in Calgary, Canada",None,154.18,None,OPTIMAL,1,Urban Core
5,949301787438025837,"Entire condo in Calgary, Canada",None,150.74,None,OPTIMAL,2,Urban Core
6,779862525321826168,"Entire rental unit in Calgary, Canada",None,132.31,None,OPTIMAL,2,Urban Core
7,53051575,"Entire condo in Calgary, Canada",None,146.21,None,OPTIMAL,2,Urban Core
8,1343745895095642454,"Entire rental unit in Calgary, Canada",None,147.28,None,OPTIMAL,2,Urban Core
9,1516380191530831771,"Entire rental unit in Calgary, Canada",None,155.50,None,OPTIMAL,1,Urban Core


In [21]:
# Close connection
conn.close()
print("\n✓ All queries completed successfully!")
print("="*80)


✓ All queries completed successfully!
